# New Direct Inversion in Iterative Space (DIIS)

This method has been extensively used to solve self-consistent field (SCF) problems in the fields of quantum chemistry and physics. In this tutorial, we employ this method to accelerate the solving of fixed-point problems.

It should be noted that one potential issue with this method is that non-negative parameters cannot be guaranteed during optimization in the conventional DIIS approach. Although this issue can be addressed by explicitly introducing constraints to the linear combination coefficients, key concepts in DIIS, numerical issues may still arise, such as singular matrices or convergence problems.

In [ ]:
import logging

import numpy as np
from setup import prepare_argument_dict, prepare_grid_and_dens, print_results

from horton_part import (
    BasisFuncHelper,
    LinearISAWPart,
    check_pro_atom_parameters,
    compute_quantities,
    lstsq_solver_dyn,
    lstsq_solver_with_extra_constr,
    lstsq_spsolver,
)

logging.basicConfig(level=logging.ERROR, format="%(levelname)s:    %(message)s")
logger = logging.getLogger(__name__)

In DIIS method, the combination coefficients are determined by solving a least-square problem.
The default method is `lstsq_spsolver`, which computes the inversion of the matrix directly.
One can also use other least-square solver and availiable methods are `lstsq_solver_dyn`, `lstsq_solver_with_extra_constr`.

In [ ]:
def run_diis(diis_method):
    """Self-consistent solver."""
    mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "diis"
    print("*" * 80)
    print(f"Results with {diis_method.__name__}".center(80, " "))
    print("*" * 80)
    kwargs["solver_options"] = {"diis_size": 8, "lstsq_solver": diis_method}
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    print()

In [ ]:
run_diis(diis_spsolver)

In [ ]:
def run_cdiis(**solver_options):
    """Self-consistent solver."""
    mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "cdiis"
    kwargs["solver_options"] = solver_options
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    print()

In [ ]:
run_cdiis(mode="R-CDIIS", param=1e-2)

In [ ]:
run_cdiis(mode="FD-CDIIS", diis_size=5)

In [ ]:
run_cdiis(mode="AD-CDIIS", param=1e-4)

In [ ]:
run_cdiis(mode="Roothaan")